In [14]:
# imports
from os import listdir
from os.path import join
from omegaconf import OmegaConf

# 1. Check if these keywords exist in the data, if so, how often?

In [18]:
# 1.1 Get keywords


def get_keywords(path):
    with open(path, "r") as keywords_file:
        keywords_data = keywords_file.readlines()

    category_keywords = {}
    for i in range(len(keywords_data)):
        category, keywords = keywords_data[i].split(":")
        keywords = keywords.split(",")
        keywords = [keyword.strip() for keyword in keywords]
        category_keywords[category] = keywords

    return category_keywords


entity_names_path = "/Users/akhil/code/lexical_lab/companies/tempus/entity_names.txt"
category_keywords = get_keywords(entity_names_path)

In [19]:
# 1.2 For each keyword, find how many times it is repeated

data_path = "/Users/akhil/code/lexical_lab/companies/tempus/Data"

def count_and_update(category_keywords, keyword_counts, text):
  for category in category_keywords:
    for keyword in category_keywords[category]:
      keyword_counts[category][keyword] += text.count(keyword)

def get_keyword_counts(data_path):
  keyword_counts = {
      category: {keyword: 0 for keyword in category_keywords[category]}
      for category in category_keywords
  }

  file_names = listdir(data_path)

  for file_name in file_names:
      file_path = join(data_path, file_name)

      with open(file_path, "r") as text_file:
          text = text_file.read()
          count_and_update(category_keywords, keyword_counts, text)

  return keyword_counts


keyword_counts = get_keyword_counts(data_path)

In [20]:
output_path = '/Users/akhil/code/lexical_lab/companies/tempus/output'
def save_yaml(dict_, output_path):
  conf = OmegaConf.create(dict_)
  with open(join(output_path, 'keywork_counts.yaml'), 'w') as fp:
     OmegaConf.save(config=conf, f=fp.name)

save_yaml(keyword_counts, output_path)

Biomarkers er and per got unusually high numbers because "er" is part of many common words.
And I am not checking for "er" as a token, but just getting the substring count.

This is fine because I just want to get an idea of how often these keywords are used.